In [7]:
import sys
sys.path.append('../..')
from sisyphus2.tasks import rdm as rd
from sisyphus2.backend.models.basic import Basic

/Users/jasminestone/anaconda2/envs/py27tensorflow/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [8]:
rdm = rd.RDM(N_in = 2, N_out = 2, dt = 10, tau = 100, T = 2000, N_batch = 128)
gen = rdm.batch_generator()

In [11]:
params = rdm.__dict__

In [12]:
params['name'] = 'basicModel'
params['N_rec'] = 50

In [13]:
basicModel = Basic(params)

In [15]:
basicModel.build()

In [16]:
basicModel.train(gen, {'save_weights_path': '../weights/refactor_weights.npz', 'training_iters': 500})

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Optimization finished!
Model saved in file: ../weights/refactor_weights.npz


([], 12.863411903381348, 33.954501152038574)